In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import speed_of_light, Boltzmann, Planck
from functools import reduce
from scipy.integrate import solve_ivp

In [2]:
from ROSAA_func import distribution, boltzman_distribution, \
    stimulated_absorption, stimulated_emission,\
    voigt, lorrentz_fwhm, gauss_fwhm

In [3]:
import pprint
pp = pprint.PrettyPrinter()
pprint = pp.pprint

In [225]:
def lineshape_normalise():

    freq = float(main_parameters["freq"])  # transition frequency in Hz

    # doppler line width

    massIon = float(lineshape_conditions["IonMass(amu)"])
    tempIon = float(lineshape_conditions["IonTemperature(K)"])
    sigma = gauss_fwhm(freq, massIon, tempIon)

    # power broadening
    dipoleMoment = float(power_broadening["dipoleMoment(D)"])
    power = float(power_broadening["power(W)"])

    cp = float(power_broadening["cp"])
    gamma = lorrentz_fwhm(dipoleMoment, power, cp)

    # normalised line shape factor
    LineShape = voigt(gamma, sigma)
    
    # transition rate due to influence of mm-wave 
    # normalisation factor

    trap_area = float(main_parameters["trap_area"])
    norm = (power/(trap_area*speed_of_light))*LineShape
    
    print(f"{massIon=}\n{tempIon=}\n{sigma=:.2e}\n{gamma=:.2e}\n{LineShape=:.2e}\n{norm=:.2e}\n")
    return norm


k_boltzmann_wavenumber = Boltzmann/1.98630e-23
def distribution(j0, j1, Energy, temp, q=None):
    
    KT = k_boltzmann_wavenumber*temp
    N0 = 2*j0+1
    N1 = 2*j1+1

    Gj = N1/N0
    delE = abs(Energy[j0]-Energy[j1])
    if q: return q*Gj*np.exp(-delE/KT)
    return Gj*np.exp(-delE/KT)

def getCollisionalRate(collisional_rates):
    
    rates = {}
    
    for i in range(totallevel):
        for j in range(totallevel):
            if i != j & j>i:
                deexciteRateConstantKey = f"q_{j}{i}"
                exciteRateConstantKey = f"q_{i}{j}"
                
                if q_deexcitation_mode:
                    
                    _temp = collisional_rates[deexciteRateConstantKey]
                    rates[deexciteRateConstantKey] = _temp
                    rates[exciteRateConstantKey] = _temp * distribution(i, j, Energy, trapTemp)
                    
                else:
                    _temp = collisional_rates[exciteRateConstantKey]
                    rates[exciteRateConstantKey] = _temp
                    rates[deexciteRateConstantKey] = _temp * distribution(j, i, Energy, trapTemp)
    return rates

In [156]:
for i in range(totallevel):
    for j in range(totallevel):
        if i != j & j>i:
            deexciteRateConstantKey = f"q_{j}{i}"
            exciteRateConstantKey = f"q_{i}{j}"
            print(f"{deexciteRateConstantKey=}\n{exciteRateConstantKey=}\n")

deexciteRateConstantKey='q_10'
exciteRateConstantKey='q_01'

deexciteRateConstantKey='q_20'
exciteRateConstantKey='q_02'

deexciteRateConstantKey='q_21'
exciteRateConstantKey='q_12'



In [284]:
def computeAttachmentProcess(N, N_He, dR_dt):
    
    # attachmentRate0, attachmentRate1, attachmentRate2 = get_attachment_rates(N, N_He)
    # Adding rare gas atom attachment and dissociation rates

    k31_0, k32 = [float(i.strip()) for i in rate_coefficients["k3"].split(",")]
    a = float(rate_coefficients["a"])
    k31_1 = a*k31_0

    kCID1, kCID2 = [float(i.strip()) for i in rate_coefficients["kCID"].split(",")]

    nHe = float(rate_coefficients["He density(cm3)"])
    Rate_k31_0 = k31_0*nHe**2
    Rate_k31_1 = k31_1*nHe**2
    Rate_k32 = k32*nHe**2
    Rate_kCID1 = kCID1*nHe
    Rate_kCID2 = kCID2*nHe

    p = float(rate_coefficients["branching-ratio"])

    attachmentRate0 = - Rate_k31_0*N[0] + Rate_kCID1*N_He[0]*p
    attachmentRate1 = - Rate_k31_1*N[1] + Rate_kCID1*N_He[0]*(1-p)
    attachmentRate2 = - Rate_k32*N_He[0] + Rate_kCID2*N_He[1]

    dR_dt[0] += attachmentRate0
    dR_dt[1] += attachmentRate1

    # CDHe:
    dCDHe_dt = - attachmentRate0 - attachmentRate1 + attachmentRate2
    dR_dt.append(dCDHe_dt)

    # CDHe2
    dCDHe2_dt = - attachmentRate2
    dR_dt.append(dCDHe2_dt)

    return dR_dt

def computeCollisionalProcess(i, N):
    
    collections = []

    for j in range(totallevel):
        if i!= j: 
            
            key = f"q_{j}{i}"
            keyInverse = f"q_{i}{j}"
            
            k = collisional_rates[key]*nHe*N[j] - collisional_rates[keyInverse]*nHe*N[i]
            collections.append(k)
            
    return collections

def computeEinsteinProcess(i, N, B_rate):
    
    collections = []
    temp = 0
    if includeSpontaneousEmission:

        # Einstein Coefficient A
        if i == 0: 
            temp = A_10*N[1]
            collections.append(temp)

        if i == 1:
            temp = -A_10*N[1]
            collections.append(temp)

    # Einstein Coefficient B

    if lightON:
        
        # NOTE: B rate defined from excited state
        
        if i==excitedLevel-1:
            temp -= B_rate
            collections.append(temp)

        if i==excitedLevel:
            temp += B_rate
            collections.append(temp)
    
    return collections

In [105]:
def computeRateDistributionEquations(t, counts):
    
    if testMode: print(f"{counts=}\n")
    
    if includeAttachmentRate:
        N =  counts[:-2]
        N_He = [counts[-2], counts[-1]]
    else:
        N = counts
    
    # B_rate defined from excited state 
    B_rate = B_01*N[excitedLevel-1] - B_10*N[excitedLevel]
    
    rateCollection = []
    
    for i in range(totallevel):
        
        collisional_collection = computeCollisionalProcess(i, N)
        einstein_collection = computeEinsteinProcess(i, N, B_rate)
        
        collections = collisional_collection + einstein_collection
        rateCollection.append(collections)
        
    dR_dt = []
    for _ in rateCollection:
        _temp = reduce(lambda a, b: a+b, _)
        dR_dt.append(_temp)
        
    if includeAttachmentRate:
        dR_dt = computeAttachmentProcess(N, N_He, dR_dt)
    
    return dR_dt

In [285]:
conditions={'trapTemp': 5.7, 'variable': 'time', 'variableRange': '1e12,  1e16,  10', 'includeCollision': True, 'writefile': True, 'filename': 'ROSAA_modal_CD_He', 
            'currentLocation': 'Z:\\Students\\Aravindh\\Measurements\\CO+', 'q_deexcitation_mode': True, 
            'collisional_rates': {'q_10':4.129378677912417e-11, 'q_20': 3.9250811501906335e-11, 'q_21': 1.2457658629399757e-10, \
                                  'q_30': 1.3300641305139243e-11, 'q_31': 8.492073041429557e-11, 'q_32': 6.81110895320521e-11}, 
            'main_parameters': {'molecule': 'CD', 'tagging partner': 'He', 'freq': '453_521_850_000', 'trap_area': '5e-5', 'Energy': '0, 15.127861, 45.373851, 90.718526, 151.132755, 226.577764'}, 
            'simulation_parameters': {'totalIonCounts': 1000, 'Simulation time(ms)': 600, 'Total steps': 1000, 'numberOfLevel (J levels)': 3, 'excitedLevel':1}, 
            'einstein_coefficient': {'A_10':'6.24e-4', 'spontaneous_emissions': [], 'includeSpontaneousEmissionForAllLevels':False, "includeSpontaneousEmission":False}, 
            'power_broadening': {'cp': '4.9e7', 'dipoleMoment(D)': 0, 'power(W)': '2e-5'}, 
            'lineshape_conditions': {'IonMass(amu)': 14, 'IonTemperature(K)': 12}, 
            'rate_coefficients': {'branching-ratio': 0.5, 'a': 0.5, 'includeAttachmentRate':False, 'He density(cm3)': '5e14', 'k3': '9.6e-31,  2.9e-30', 'kCID': '6.7e-16,  1.9e-15'}
        }

main_parameters = conditions["main_parameters"]
simulation_parameters = conditions["simulation_parameters"]
einstein_coefficient = conditions["einstein_coefficient"]
lineshape_conditions = conditions["lineshape_conditions"]
rate_coefficients = conditions["rate_coefficients"]
power_broadening = conditions["power_broadening"]

totallevel = simulation_parameters["numberOfLevel (J levels)"]



Energy = [float(_) for _ in main_parameters["Energy"].split(", ")][:totallevel]
trapTemp = conditions["trapTemp"]

excitedLevel = simulation_parameters["excitedLevel"]

A_10 = float(einstein_coefficient["A_10"])
includeSpontaneousEmissionForAllLevels = einstein_coefficient["includeSpontaneousEmissionForAllLevels"]

freq = float(main_parameters["freq"])  # transition frequency in Hz
norm = lineshape_normalise()

B_10 = stimulated_emission(A_10, freq)*norm
B_01 = stimulated_absorption(excitedLevel-1, excitedLevel, B_10)*norm

excitedLevel = simulation_parameters["excitedLevel"]
includeAttachmentRate = rate_coefficients["includeAttachmentRate"]
print(f"{includeAttachmentRate=}\n{includeCollision=}\n{includeSpontaneousEmission=}")

collisional_rates = {q:float(value) for q, value in conditions["collisional_rates"].items()}
q_deexcitation_mode = conditions["q_deexcitation_mode"]

collisional_rates = getCollisionalRate(collisional_rates)
collisional_rates

massIon=14.0
tempIon=12.0
sigma=3.00e+05
gamma=2.19e+05
LineShape=8.08e-07
norm=1.08e-15

includeAttachmentRate=False
includeCollision=True
includeSpontaneousEmission=False


{'q_10': 4.129378677912417e-11,
 'q_01': 2.7212068747455605e-12,
 'q_20': 3.9250811501906335e-11,
 'q_02': 2.0852215354513816e-15,
 'q_21': 1.2457658629399757e-10,
 'q_12': 1.004299251135604e-13}

In [286]:
N = boltzman_distribution(Energy, 300)
N_He = [0, 0]
lightON=True

testMode = True
dN = computeRateDistributionEquations(1, [*N, 0, 0])
dN

counts=[0.12800280612319145, 0.35713638764020317, 0.5148608062366054, 0, 0]



[21471.894693248905, 20684.19131573619, -42156.08600898509]

array([0.13701834, 0.36867186, 0.49430979])

In [338]:
%%time

# nHe = float(rate_coefficients["He density(cm3)"])
# includeSpontaneousEmission = einstein_coefficient["includeSpontaneousEmission"]
# includeCollision = conditions["includeCollision"]
nHe = float(rate_coefficients["He density(cm3)"])
includeSpontaneousEmission = True
includeCollision = True
includeAttachmentRate = True

testMode = False
t = 1000e-3
tspan = [0, t]
N = boltzman_distribution(Energy, 200)
N_He = [0, 0]
boltzman_distribution_source = (N, [*N, 0, 0])[includeAttachmentRate]
print(f"{boltzman_distribution_source=}")

testMode = False
print(f"LightOFF")
lightON=False
N_rates_off = solve_ivp(computeRateDistributionEquations, tspan, boltzman_distribution_source, dense_output=True)

print(N_rates_off.sol(simulateTime).T[-1])
if not includeAttachmentRate:
    %matplotlib widget
    simulateTime = np.linspace(0, t, 1000)
    simulateCounts_OFF = N_rates_off.sol(simulateTime)

    fig, ax = plt.subplots(figsize=(7, 4), dpi=100)
    legends = [f"N{i}" for i in range(totallevel)]

    ax.plot(simulateTime.T*1e3, simulateCounts_OFF.T)

    ax.plot(simulateTime*1e3, simulateCounts_OFF.sum(axis=0), "k")
    ax.legend(legends, title="OFF")

    ax.set(yscale="linear", xlabel="Time (ms)")

boltzman_distribution_source=[0.1370183412076079, 0.36867186391379997, 0.49430979487859217, 0, 0]
LightOFF
[8.25911529e-01 5.44278448e-02 4.35387856e-05 9.96892583e-02
 1.99278295e-02]
Wall time: 7.44 s


In [339]:
%%time
print(f"LightON")
lightON=True
N_rates_on = solve_ivp(computeRateDistributionEquations, tspan, boltzman_distribution_source, dense_output=True)

LightON
Wall time: 7.72 s


In [340]:
%matplotlib widget
simulateTime = np.linspace(0, t, 1000)
simulateCounts_ON = N_rates_on.sol(simulateTime)
simulateCounts_OFF = N_rates_off.sol(simulateTime)

print(f"{simulateCounts_ON.shape=}\n{simulateCounts_OFF.shape=}")
fig, ax = plt.subplots(figsize=(7, 4), dpi=100)

legends = [f"N{i}" for i in range(totallevel)]
legends.append("NHe")
legends.append("NHe2")

counter = 0
for on, off in zip(simulateCounts_ON, simulateCounts_OFF):
    ax.plot(simulateTime, on, f"-C{counter}", label=legends[counter])
    ax.plot(simulateTime, off, f"--C{counter}")
    counter += 1
    
ax.plot(simulateTime, simulateCounts_ON.sum(axis=0), "k")
ax.legend(title=f"-ON, --OFF")
ax.set(yscale="log")

simulateCounts_ON.shape=(5, 1000)
simulateCounts_OFF.shape=(5, 1000)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[None]